In [ ]:
import glob
import os
import numpy as np
import random
import re 
# I rename the class VTKObject as Mesh so I don't have to change the function calls in the code
from VTK.VTKMesh import VTKObject as Mesh  #, MeshViewer, MeshViewers
import meshio
import time
from copy import deepcopy
import random
from sklearn.decomposition import PCA
from tqdm import tqdm # for the progress bar

In [ ]:
class NumpyFromVTKs(object):
    
    """docstring for NumpyFromVTKs"""

    def __init__(self, folders, filename_pattern="*/output.001.vtk", dataset_name="LV_endo_100", partition_ids=None, subj_ids=None, N_subj=None):
        
        '''          
          - subj_ids: list of subject IDs to draw samples from (if None, it's all the subjects)
          - N_subj: number of subjects to sample (if None, it's all the subjects from subj_ids)
        '''
        
        self.folders = folders if isinstance(folders, list) else [folders]
        self.filename_pattern = os.path.join(folders, filename_pattern)
        self.regex = re.compile(self.filename_pattern.replace("*","(.*)"))
        self.dataset_name = dataset_name # output directory
        
        self.partition_ids = partition_ids
        self.N_subj = N_subj
        self.subj_ids = subj_ids 

        self.gather_paths()
        # self.train_vertices = self.gather_data(self.datapaths["train"])
        # self.test_vertices = self.gather_data(self.datapaths["test"])

        #self.save_vertices()

        
    def extract_id_from_path(self, path):
            # compile once above
            id = self.regex.match(path).group(1)
            return id
        
        
    def gather_paths(self, test_fraction=0.1):
        
        '''
        :param opt: 'train' or 'test'
        :return:
        '''

        datapaths = []
        for subdir_name in self.folders:
            datapaths.extend(sorted(glob.glob(self.filename_pattern)))
            
        print(len(datapaths))
            
        if self.subj_ids is None:            
            id_dict = {self.extract_id_from_path(x): x for x in datapaths}
        else:
            id_dict = {self.extract_id_from_path(x): x for x in datapaths if self.extract_id_from_path(x) in subj_ids}
            print(len(id_dict))
            
        
        if self.N_subj is not None:
            ind_list = list(range(len(id_dict)))
            random.shuffle(ind_list)
            ind_list = ind_list[0:self.N_subj]
            datapaths = [id_dict[i] for i in ind_list]
        else:
            datapaths = id_dict.values()
            pass # use all the data paths
                                        
        self.datapaths = datapaths


    def gather_data(self):
        vertices = []

        # tqdm: for progress bar (I think)
        for p in tqdm(self.datapaths, unit="subjects"):
            mesh_filename = p
            mesh = Mesh(filename=mesh_filename) # Load mesh
            mesh = Mesh.extractSubpart(mesh, self.partition_ids)
            vertices.append(mesh.points)

        return np.array(vertices)


    def save_vertices(self):

        if not os.path.exists(self.dataset_name):
            os.makedirs(self.dataset_name)

        return 0

In [ ]:
kk = NumpyFromVTKs(
  folders = "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/vtk_meshes", 
  filename_pattern="*/output.001.vtk", 
  dataset_name="", subj_ids=None, N_subj=None, partition_ids=[1,2]
)

In [1]:
import cardiac_mesh

In [2]:
cardiac_mesh.generateNumpyDataset("/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/vtk_meshes", "/MULTIX/DATA/INPUT/disk_2/coma/Cardio/meshes/numpy_files/LV_all_subjects")

100%|██████████| 100/100 [02:46<00:00,  1.66s/subjects]


0